# Mining words from Wikipedia

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import array_contains
import unicodedata
import pycountry

spark = SparkSession \
    .builder \
    .appName("Analysing Wikipedia") \
    .getOrCreate()

In [6]:
df = spark.read.json("./nowiki-20210111-cirrussearch-content.json")

## Cleaning the dataset

Looking at the schema just to explore the dataset. Found [a description of the JSON dump format on Wikipedia](https://meta.wikimedia.org/wiki/Data_dumps/Misc_dumps_format)

In [7]:
df.printSchema()

root
 |-- auxiliary_text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- category: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- content_model: string (nullable = true)
 |-- coordinates: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- coord: struct (nullable = true)
 |    |    |    |-- lat: double (nullable = true)
 |    |    |    |-- lon: double (nullable = true)
 |    |    |-- country: string (nullable = true)
 |    |    |-- dim: long (nullable = true)
 |    |    |-- globe: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- primary: boolean (nullable = true)
 |    |    |-- region: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- create_timestamp: string (nullable = true)
 |-- defaultsort: string (nullable = true)
 |-- display_title: string (nullable = true)
 |-- external_link: array (nullable = true)
 |    |-- element: strin

In [8]:
df.show()

+--------------------+--------------------+-------------+--------------------+--------------------+----------------+-------------+--------------------+--------------------+--------------+--------------+--------+---------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------+------------+------+-------------+
|      auxiliary_text|            category|content_model|         coordinates|    create_timestamp|     defaultsort|display_title|       external_link|             heading|incoming_links|         index|language|namespace|namespace_text|        opening_text|   ores_articletopic|  ores_articletopics|       outgoing_link|    popularity_score|            redirect|             score|         source_text|            template|                text|text_bytes|    

### Find columns to filter on

In [9]:
# Should one filter out null?
df.select("content_model").distinct().show()

+-------------+
|content_model|
+-------------+
|         null|
|     wikitext|
+-------------+



In [10]:
df.filter(df["content_model"].isNotNull()).count()

548219

In [11]:
df.filter(df["content_model"].isNull()).count()

548219

In [12]:
# It seems like every other row in this dataset consists of just nulls,
# so I remove those rows
df_not_null = df.filter(df["content_model"].isNotNull())

In [13]:
# After removing the nulls, there is only one language (where previously null also showed up)
df_not_null.select("language").distinct().show()

+--------+
|language|
+--------+
|      nb|
+--------+



In [14]:
# About namespaces https://en.wikipedia.org/wiki/Wikipedia:Namespace
# Articles have no namespace (no prefix), we are interested namespace 0.
# It seems this dataset only contains the namespace we are interested in!
df_not_null.select("namespace").distinct().show()

+---------+
|namespace|
+---------+
|        0|
+---------+



### Filtering out columns

In [15]:
filtered_df = df_not_null.drop("content_model", "language", "category", "coordinates", "defaultsort", \
        "external_link", "heading", "incoming_links", "namespace", "namespace_text", \
        "outgoing_link", "redirect", "text_bytes", "template", "wiki", \
        "wikibase_item", "version_type", "file_bits", "file_height", "file_media_type", \
        "file_resolution", "file_size", "file_text", "file_width", "index", \
        "file_mime", "ores_articletopic", "ores_articletopics", "score", "popularity_score", \
        "display_title", "auxiliary_text", "create_timestamp", "opening_text", "source_text", \
        "timestamp", "version")

In [16]:
# Seems like only one entry per article. That's good! I feared it was one per revision.
filtered_df.filter(filtered_df["title"] == "Kill Buljo").show()

+--------------------+----------+
|                text|     title|
+--------------------+----------+
|Kill Buljo er en ...|Kill Buljo|
+--------------------+----------+



In [17]:
# Comparing the content in the text column to the content on the web page
# https://no.wikipedia.org/wiki/Kill_Buljo
filtered_df.filter(filtered_df["title"] == "Kill Buljo").select("text").collect()[0][0]

'Kill Buljo er en norsk film fra 2008. Den er en komisk parodi på det amerikanske actioneeventyret Kill Bill, og handlingen er lagt til Finnmark. Regi er ved Tommy Wirkola og manus Stig Frode Henriksen og Tommy Wirkola. Filmen hadde premiere i mars 2007, og ble sett av 87 000 på kino i Norge. Det ble solgt over 95 000 DVD-er. Filmmusikken ble bl.a. laget av Alta-bandet Cyaneed. Kill Buljo 2 kom ut i 2013. Jompa Tormann er rollefiguren som spilles av Stig Frode Henriksen i filmen Kill Buljo. Rollefiguren dukker også opp i flere kortfilmer på DVD-en Kill Buljo: The Beginning. Jompa Tormann: Stig Frode Henriksen Pappa Buljo: Frank Arne Olsen Tampa Buljo: Martin Hykkerud Sid Wisløff: Tommy Wirkola Unni Formen: Natasha Angel Dahle Peggy Mathilassi: Linda Øverlie Nilsen Crazy Beibifeit: Ørjan Gamst Kjell Driver: John Even Pedersen Bud Light: Christian Reiertsen Lara Kofta: Merete Nordahl Mr. Handjagi: Ørjan Gamst Kato: Jørn Tore Nilsen Blow Job: Heidi Monsen Troll Tove: Eirik Junge Eliassen 

It seems the text column contains everything in the article, so I went back and removed all columns except `text` and `title`.

The text content itself could need some cleaning. There are several instances of `(en)` which looking at the web page seems to be the language of the external reference, so at least remove `(en)` and `(no)`. At one point we see `[død lenke]` which is an inline footnote, possibly a generic one. Going to the web page we find that it was created by a bot. On Wikipedia we find a web page listing [all referance template tags that can appear in running text](https://no.wikipedia.org/wiki/Mal:Trenger_referanse).

Also found a [list over sentences called "stub"](https://no.wikipedia.org/wiki/Kategori:Stubbmaler) that can appear in the running text

We see that \xa0 appears.

And lastly we want to remove punctuation marks, digits and other special characters. Basically be left with only words consisting of letters.

### Cleaning the text content

In [46]:
possible_template_reference_tags = ["[trenger referanse]", "[klargjør]", "[hvor?]", "[trenger oppdatering]", "[død lenke]", "[trenger sitat]", "[trenger bedre kilde]", "[bør utdypes]", "[hvem?]", "[omstridt – diskuter]", "[ufullstendig referanse]", "[ikke i angitt kilde]", "[tredjepartskilde trengs]", "[når?]", "[av hvem?]", "[sic]", "[trenger sidetall]"]

# First time I understand this is a generator and not just a for loop
# Weirdly enough, the ISO 639-3 alpha_2 codes doesn't contain "en"
ext_ref_language_tags = [f"({country.alpha_2.lower()})" for country in pycountry.countries] + ["(en)"]

# Maybe I should automate this... it is 361 entries! https://no.wikipedia.org/wiki/Kategori:Stubbmaler
possible_template_stub_tags = [
    "Denne artikkelen er foreløpig kort eller mangelfull. Du kan hjelpe Wikipedia ved å utvide eller endre den.",
    "Denne seksjonen er en stubb. Du kan bidra med å utvide den.",
    "Denne musikkalbum-relaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den.",
    "Denne alternativ rock-relaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den.",
    "Denne anatomirelaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den.",
    "Denne romerskrelaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den.",
    "Denne astronomirelaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den.",
    "Denne avisrelaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den.",
    "Denne bilrelaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den.",
    "Denne biokjemirelaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den.",
    "Denne biologirelaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den.",
    "Denne botanikkrelaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den.",
    "Denne datarelaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den.",
    "Denne drikkerelaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den.",
    "Denne elektrorelaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den.",
    "Denne farmakologirelaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den.",
    "Denne festivalrelaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den.",
    "Denne filmrelaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den.",
    "Denne filosofirelaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den.",
    "Denne fjellrelaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den. Det finnes en mer utfyllende artikkel om emnet på nynorsk."
    "Denne fjellrelaterte artikkelen er foreløpig kort eller mangelfull, og du kan hjelpe Wikipedia ved å utvide den."

]

In [19]:
# Use RDD features to map and reduce
# Transform from Row with title and text to just text strings, rigth away
rdd = filtered_df.rdd.map(lambda row: row.text)

In [21]:
def remove_strings(src_string: str, strings_to_remove: list):
    new_string= src_string
    for s in strings_to_remove:
        new_string = new_string.replace(s, "")
    return new_string

In [47]:
removed_templates_rdd = rdd.map(lambda s: remove_strings(s, possible_template_reference_tags))
removed_templates_rdd.take(2)

['Hundene i Riga er en svensk film fra 1995 av Per Berglund med Rolf Lassgård, Björn Kjellman, Charlotte Sieling og Paul Butkevich i noen av rollene. Filmen basert seg på romanen Hundene i Riga av Henning Mankell som er den andre i serien om Kurt Wallander. En livbåt flyter i land ved den skånske kysten. I båten befinner det seg to menn som har blitt myrdet. Etterforsker Kurt Wallander fra politiet i Ystad tilkalles til plassen. Ved hjelp av politiet i Latvia blir begge mennene identifisert. For å lette utredningen ble en politimann tilkalt fra Latvia for å hjelpe til å løse saken. Men når politimannen vender tilbake til Latvia blir han myrdet. Kurt Wallander flyr til Latvia for å forsøke å finne ut hvorfor politimannen ble myrdet. (en) Hundene i Riga på Internet Movie Database (sv) Hundene i Riga i Svensk Filmdatabas (en) Hundene i Riga på Rotten Tomatoes Portal: Film',
 'Kill Buljo er en norsk film fra 2008. Den er en komisk parodi på det amerikanske actioneeventyret Kill Bill, og ha

In [48]:
removed_lang_tags_rdd = removed_templates_rdd.map(lambda s: remove_strings(s, ext_ref_language_tags))
removed_lang_tags_rdd.take(2)

['Hundene i Riga er en svensk film fra 1995 av Per Berglund med Rolf Lassgård, Björn Kjellman, Charlotte Sieling og Paul Butkevich i noen av rollene. Filmen basert seg på romanen Hundene i Riga av Henning Mankell som er den andre i serien om Kurt Wallander. En livbåt flyter i land ved den skånske kysten. I båten befinner det seg to menn som har blitt myrdet. Etterforsker Kurt Wallander fra politiet i Ystad tilkalles til plassen. Ved hjelp av politiet i Latvia blir begge mennene identifisert. For å lette utredningen ble en politimann tilkalt fra Latvia for å hjelpe til å løse saken. Men når politimannen vender tilbake til Latvia blir han myrdet. Kurt Wallander flyr til Latvia for å forsøke å finne ut hvorfor politimannen ble myrdet.  Hundene i Riga på Internet Movie Database  Hundene i Riga i Svensk Filmdatabas  Hundene i Riga på Rotten Tomatoes Portal: Film',
 'Kill Buljo er en norsk film fra 2008. Den er en komisk parodi på det amerikanske actioneeventyret Kill Bill, og handlingen er 